In [115]:
import glob
import os
import sys
from math import sqrt
from PIL import Image
from resizeimage import resizeimage
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing
from skimage.color import rgb2xyz, xyz2rgb
import matplotlib.pyplot as plt

In [39]:
size=25 # A CHANGER
threshold = 130
folder = "PeopleArt/resized/"+str(size)+"/"
print(size)
print(folder)

25
PeopleArt/resized/25/


In [40]:
print("Step 1 - Resize ...")

if not os.path.exists(folder):
    os.makedirs(folder)

images = glob.glob("PeopleArt/JPEGImages/*/*")

i = 1
for image in images:
    label = image.split("/")[2].lower()
    with open(image, 'r+b') as f:
        img = Image.open(f)
        img = resizeimage.resize_cover(img, [size, size])
        img.save(folder+label+"_"+str(i)+'.jpg', img.format)
    i += 1

Step 1 - Resize ...


In [41]:
images = shuffle(glob.glob("PeopleArt/resized/"+str(size)+"/*"))
print("Step 2 - Convert to GreyScale ...")
images = shuffle(glob.glob(folder+"*"))
labels = np.array([])
X_data = np.array([])

for image in images:    
    labels = np.append(labels,image.split("/")[3].split("_")[0].lower())    
    image = Image.open(image).convert('L')
    arr = np.array(image)
    arr = arr.reshape(1,size*size)    
    if X_data.any() :
        X_data = np.concatenate( (X_data, arr), axis=0  )
    else:
        X_data = arr


le = preprocessing.LabelEncoder()
le.fit(labels)
num_labels = le.transform(labels)

np.savetxt('matrice-greyscale-'+str(size)+'.csv', X_data, fmt='%3d', delimiter=',')
np.savetxt('labels-'+str(size)+'.csv', num_labels, fmt='%3d', delimiter=',')
np.savetxt('annotation-'+str(size)+'.csv', labels, fmt='%3s', delimiter=',')

Step 2 - Convert to GreyScale ...


In [42]:
print("Step 3 - Convert to binary ...")

def binarize_array(numpy_array, threshold=200):
    """Binarize a numpy array."""
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] == True:
                numpy_array[i][j] = 1
            else:
                numpy_array[i][j] = 0
    return numpy_array

print("threshold : ", threshold)
images = shuffle(glob.glob("PeopleArt/resized/"+str(size)+"/*"))
X_data_bin = np.array([])
for image in images:
    image = Image.open(image).convert('1')
    arr = binarize_array(np.array(image),threshold)
    arr = arr.reshape(1,size*size)
    if X_data_bin.any() :
        X_data_bin = np.concatenate( (X_data_bin, arr), axis=0  )
    else:
        X_data_bin = arr

np.savetxt('matrice-binary-'+str(size)+'.csv', X_data_bin, fmt='%3d', delimiter=',')

Step 3 - Convert to binary ...
('threshold : ', 130)


In [160]:
print("Step 4 - Convert to size format "+str(size)+" rgb to int...")
images = shuffle(glob.glob(folder+"*"))
labels = np.array([])
X_data = np.array([])

for image in images:    
    labels = np.append(labels,image.split("/")[3].split("_")[0].lower())    
    image = Image.open(image).convert('RGB')
    arr = np.array(image)
    arr = arr.reshape(1,size*size,3)    
    if X_data.any() :
        X_data = np.concatenate( (X_data, arr), axis=0)
    else:
        X_data = arr

rgb_to_int = X_data[:, :, 0] + X_data[:, :, 1]*256 + X_data[:, :, 2]*256*256        

le = preprocessing.LabelEncoder()
le.fit(labels)
num_labels = le.transform(labels)
np.savetxt('matrice-resize-'+str(size)+'.csv', rgb_to_int, fmt='%3d', delimiter=',')

Step 4 - Convert to size format 25 ...


In [170]:
print("Step 5 - Convert to size format "+str(size)+" 3-D RGB...")
images = shuffle(glob.glob(folder+"*"))
labels = np.array([])
X_data = np.array([])

for image in images:    
    labels = np.append(labels,image.split("/")[3].split("_")[0].lower())    
    image = Image.open(image).convert('RGB')
    arr = np.array(image) 
    if X_data.any() :
        X_data = np.concatenate((X_data, arr))
    else:
        X_data = arr
    
le = preprocessing.LabelEncoder()
le.fit(labels)
num_labels = le.transform(labels)
img = Image.fromarray(X_data, 'RGB')
img.save('matrice-original-'+str(size)+'.png')

Step 5 - Convert to size format 25 3-D RGB...


In [121]:
def display_matrix(X, row=None, size=16):
    X_data = np.array([])
    if row is None:
        row = sqrt(len(X))
    for x in X:
        arr = x.reshape(size,size)
        if X_data.any() :
            X_data = np.concatenate( (X_data, arr), axis=1 )
        else:
            X_data = arr
    n = np.array_split(X_data, row, axis=1)
    X_data = np.concatenate(n, axis=1)
    plt.figure(figsize=(25, 25))
    plt.imshow(X_data)
    plt.axis("off")
    plt.show()

In [168]:
datas = (np.loadtxt(open("matrice-original-25.csv", "rb"), delimiter=",")).T #Transpose la matrice
datas

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: loadtxt: Empty input file: "<open file 'matrice-original-25.csv', mode 'rb' at 0x7fcf4b9b7f60>"
  """Entry point for launching an IPython kernel.


array([], dtype=float64)